# Aligning the science exposures

To align the science exposures and remove the shifts seen in a previous exercise we use the AstroAlign package.
AstroAlign looks for 3-point asterisms (i.e., triangles of stars) in pairs of images and computes a linear transformation based on matching asterisms.

We import the package as aa, and we will also need astropy.io.fits to read fits files:

In [ ]:
import astroalign as aa
import astropy.io.fits as fits

We now read the reference ("target") image and the image to be transformed ("source"):

In [ ]:
datatarg = fits.getdata('science1Vf.fits')
datasrc = fits.getdata('science10Vf.fits')

Finding the transformation from the source to the target frame requires only a single call to find_transform():

In [ ]:
T, (source_pos_array, target_pos_array) = aa.find_transform(datasrc, datatarg)

The object T has attributes that contain information about the rotation, translation, and scaling of the transformation (T.rotation, T.translation, and T.scale). For a sequence of consecutive observations, we expect the rotation to be close to 0 and the scale should be close to 1 for any pair of observations obtained with the same instrument (no scale change). We do expect shifts (translations) due to inaccurate tracking. Let us check:

In [ ]:
print(T.scale)

In [ ]:
import math
print(T.rotation*180/math.pi)

So in this case, the scale and rotation are indeed small. To get an idea about what is acceptable, we recall that the longest dimension of the CCD image is 2184 pixels:

In [ ]:
datatarg.shape

Hence, the rotation corresponds to a shift of much less than one pixel from one side of the CCD to the other:

In [ ]:
datatarg.shape[1] * T.rotation

Next, we check the shift, which may be larger (e.g. due to guiding / tracking errors). This is what we need to correct.

In [ ]:
print(T.translation)

The source_pos_array and target_pos_array variables are lists of the coordinates of the sources used by astroalign to define the transformation.  Let us plot them on top of the image to check that astroalign has made a sensible choice of stars to use for the transformation.

First, we transpose the target_pos_array to two individual arrays containing the x and y coordinates. This is because the coordinates are stored as an array of [x, y] pairs in target_pos_array, whereas we need the x and y in separate arrays:

In [ ]:
print(target_pos_array)

Luckily, doing the conversion is easy. We just use the transpose() method:

In [ ]:
x, y = target_pos_array.transpose()
print(x)
print(y)

Now we can plot the coordinates of the stars used by AstroAlign on top of the target image:

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(datatarg, vmin=500, vmax=1500, cmap='magma')
plt.plot(x, y, 'wo', fillstyle='none', markersize=10)

Everything seems OK, so we go ahead and apply the transformation to the source image:


In [ ]:
data_tran, footprint = aa.apply_transform(T, datasrc, datatarg)

and then write the result to an output file

In [ ]:
fits.writeto('science10Vft.fits', data_tran, overwrite=True)

Our final test of whether the transformation went well is to blink the two images against each other. We can do this in DS9. Let us see how we can do all of this via the imexam interface. First we set up a viewer interface and load the reference image:

In [ ]:
import imexam
viewer = imexam.connect()

In [ ]:
viewer.load_fits('science1Vf.fits')
viewer.scale()

Then we open a second frame in DS9 and load the transformed source image:

In [ ]:
viewer.frame(2)
viewer.load_fits('science10Vft.fits')

And then we blink the two frames:

In [ ]:
viewer.blink()

Hopefully, any shifts will now have disappeared. 

## If it fails

Occasionally, the procedure described above may randomly fail if an insufficient number of stars are identified and used for the transformation.

It is therefore extremely important to check that the rotation and scale change are within acceptable limits. 
If this is not the case, then the first thing to try is to just call find_transform() again. 

If the problem persists and find_transform() is unable to find an acceptable transformation after repeated attempts, an alternative solution is to manually define a list of reference coordinates. This can then be passed to find_transform() instead of the images. The rest of the procedure works as before.

Let us see how a transformation can be defined using the coodinates of stars in science1V.FIT and science10V.FIT that we measured earlier:

In [ ]:
import numpy as np
coo_targ = np.loadtxt('coo1V.txt', usecols=(0,1))
coo_src  = np.loadtxt('coo10V.txt', usecols=(0,1))

In [ ]:
T, (source_pos_array, target_pos_array) = aa.find_transform(coo_src, coo_targ)

In [ ]:
print(T.scale, T.rotation, T.translation)

- Compare the shifts determined by find_transform() with those determined in Assignment 4.1

In order to get a reliable transformation, it is important to select stars that are distributed across the image, so that the scale and rotation are well constrained.